In [ ]:
import sqlite3 as sql
import pandas as pd

import plotly
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as gro
from plotly.subplots import make_subplots


#### Delcaring Globals

In [ ]:
pio.renderers.default = 'browser'

CountryList = [];
InternetAccessList = [];
LifeSatisfactionList = [];
YearList = [];

Setting up Database Connector and Querying All Availible Countries

In [ ]:
Connection = sql.connect('../Raw Data/HOORAH.db');

Query = 'SELECT * FROM Countries';
Data = pd.read_sql_query(Query, Connection);
TotalCountryList = Data['Country Name'].to_numpy();

for i in range(len(TotalCountryList)):
    Query = f'SELECT * FROM "{TotalCountryList[i]}"';
    CountryData = pd.read_sql_query(Query, Connection);

    for j in range(len(CountryData['Year'])):
        if(CountryData['Internet Access'][j] and CountryData['Satisfaction'][j]):
            CountryList.append(TotalCountryList[i]);
            InternetAccessList.append(CountryData['Internet Access'][j]);
            LifeSatisfactionList.append(CountryData['Satisfaction'][j]);
            YearList.append(j + 2014);

Creating a joint Pandas Dataframe and Sorting Years in ascending order

In [ ]:
Data = pd.DataFrame({'Country' : CountryList, 'Year' : YearList, 'Internet Access' : InternetAccessList,
                     'Life Satisfaction' : LifeSatisfactionList});
Data = Data.sort_values(by=['Year']);

Creating our Graph subplots and adding a choropleth and scatterplot

In [ ]:
Figure = make_subplots(rows=1, cols=2, specs=[[{'type' : 'choropleth'}, {'type' : 'scatter'}]]);

for i in range(4):
    tempData = Data[Data.Year == (2014 + i)];
    print(tempData);
    isVisible = True;

    if(i):
        isVisible = False;

    Figure.add_choropleth(z=tempData['Internet Access'].to_numpy(), name='Internet Access', colorscale='viridis',
                          colorbar_x=0, colorbar_title_text="Life Satisfaction (%)", colorbar_title_side= "right", colorbar_title_font_size=30,
                          zmax=100, zmin=70, locationmode='country names', locations=tempData['Country'].to_numpy(),
                          visible=isVisible, row=1, col=1);
    Figure.update_geos(scope = "europe")
    #Figure.add_choropleth(z=tempData['Life Satisfaction'].to_numpy(), name='Life Satisfaction', locationmode='country names', locations=tempData['Country'].to_numpy(), visible=isVisible, row=1, col=2);

    Figure.add_scatter(x=tempData['Internet Access'], y=tempData['Life Satisfaction'], name='Internet Access vs Life Satisfaction',
                       visible=isVisible, mode='markers', row=1, col=2);
    Figure.update_xaxes(range=[70,100])
    Figure.update_yaxes(range=[4,10])

             Country  Year  Internet Access  Life Satisfaction
0            Albania  2014            90.87                5.8
36            Greece  2014            85.07                6.8
40             Spain  2014            95.92                7.3
44           Finland  2014            96.56                8.0
48            France  2014            93.33                6.7
52           Croatia  2014            86.09                6.9
56           Hungary  2014            90.78                6.5
60           Ireland  2014            97.50                7.2
128   United Kingdom  2014            94.85                7.6
65             Italy  2014            90.49                7.3
69         Lithuania  2014            86.55                6.7
73        Luxembourg  2014            99.18                7.5
77            Latvia  2014            91.15                6.5
81        Montenegro  2014            80.76                6.5
84   North Macedonia  2014            79.32            

makes final graph with time slider

In [ ]:
steps = []
for i in range(4):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(Figure.data)],
        label = f'Year {2014 + i}'
    )

    step['args'][1][i * 2] = True
    step['args'][1][i * 2 + 1] = True
    steps.append(step)

sliders = [dict(
    steps = steps,
)]

Figure.update_layout(
    xaxis_title = 'Internet Access',
    xaxis_title_font_size= 20,
    yaxis_title = 'Life Satisfaction',
    yaxis_title_font_size= 20,
    )

Figure.layout.sliders = sliders;
Figure.show();